# Getting started

In [1]:
import pandas as pd
import numpy as np
import random

import akiFlagger

In [2]:
akiFlagger.__version__

'0.0.3'

### Let's start off by creating some toy data

In [3]:
df = akiFlagger.generate_toy_data()
df.head()

Successfully generated toy data!


/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimelike.py:1215: PerformanceWarning: Adding/subtracting array of DateOffsets to DatetimeArray not vectorized
  PerformanceWarning,


,mrn,enc,admission,time,creat
0,12732,25741,2020-02-27 11:42:42,2020-02-27 11:42:42,1.116895
1,19845,81382,2020-05-12 06:02:54,2020-05-08 18:02:54,0.390540
2,13264,89464,2020-01-10 11:16:57,2020-01-14 05:16:57,1.353141
3,14859,75180,2020-06-07 12:27:38,2020-06-09 00:27:38,1.182348
4,19225,94917,2020-03-26 21:16:49,2020-03-26 21:16:49,1.232916


In [4]:
rw = akiFlagger.returnAKIpatients(df, aki_calc_type = 'rolling_window')
rw.head()

mrn    enc           admission     creat  \
mrn   time                                                              
12732 2020-02-22 17:42:42  12732  65270 2020-02-27 11:42:42  1.359735   
      2020-02-24 23:42:42  12732  45050 2020-02-27 11:42:42  1.536899   
      2020-02-25 17:42:42  12732  15103 2020-02-27 11:42:42  0.842858   
      2020-02-26 17:42:42  12732  67368 2020-02-27 11:42:42  1.277589   
      2020-02-27 11:42:42  12732  25741 2020-02-27 11:42:42  1.116895   

                           mincreat_48hr  mincreat_7day  deltacreat_48hr  \
mrn   time                                                                 
12732 2020-02-22 17:42:42       1.359735       1.359735          0.00000   
      2020-02-24 23:42:42       1.536899       1.359735          0.00000   
      2020-02-25 17:42:42       0.842858       0.842858          0.00000   
      2020-02-26 17:42:42       0.842858       0.842858          0.43473   
      2020-02-27 11:42:42       0.842858       0.842858          0.27404   

                           deltacreat_7day  rollingwindow_aki  stage1  stage2  \
mrn   time                                                                      
12732 2020-02-22 17:42:42          0.00000              False   False   False   
      2020-02-24 23:42:42          0.17716              False   False   False   
      2020-02-25 17:42:42          0.00000              False   False   False   
      2020-02-26 17:42:42          0.43473               True    True   False   
      2020-02-27 11:42:42          0.27404              False   False   False   

                           stage3  
mrn   time                         
12732 2020-02-22 17:42:42   False  
      2020-02-24 23:42:42   False  
      2020-02-25 17:42:42   False  
      2020-02-26 17:42:42   False  
      2020-02-27 11:42:42   False

In [5]:
df = akiFlagger.generate_toy_data(include_demographic_info = True)
df.head()

Successfully generated toy data!


/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimelike.py:1215: PerformanceWarning: Adding/subtracting array of DateOffsets to DatetimeArray not vectorized
  PerformanceWarning,


,mrn,enc,age,black,female,admission,time,creat
0,12732,25741,59.6306,False,False,2020-02-27 11:42:42,2020-02-22 11:42:42,1.490517
1,19845,81382,57.67,False,False,2020-05-12 06:02:54,2020-05-16 12:02:54,0.367723
2,13264,89464,69.4009,False,True,2020-01-10 11:16:57,2020-01-09 05:16:57,1.297591
3,14859,75180,64.9212,True,True,2020-06-07 12:27:38,2020-06-10 00:27:38,1.049496
4,19225,94917,67.1327,False,True,2020-03-26 21:16:49,2020-03-24 09:16:49,0.784172
